In [1]:
from PIL import Image
import glob
import numpy as np
import os
from os.path import basename

image_Lbl_list = []
for filename in glob.glob('Dataset/*.png'): #assuming png
    im = Image.open(filename)
    label = (filename.split("_")[1])
    #print(label)
    image_Lbl_list.append(label)
    im.close()

#print(image_Lbl_list)

#path = basename('Images/0')
#path

In [3]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

flat_Image_List = [] # new list containing flatten image arrays
#for img in image_Lbl_list: # iterate over the images contained in the directory
for filename in glob.glob('Dataset/*.png'):
    im = Image.open(filename)
    img = np.reshape(im, (1, 49152)) # flattened each imag (3 per each pixel)
    flat_Image_List.append(img) # Append to the list 
    #plt.imshow(img)
    im.close()
    #print(flat_Image_List)

#print(flat_Image_List)

In [4]:
from numpy import array

#flat_Image_Array = array(flat_Image_List)
flat_Image_Array = flat_Image_List.reshape(flat_Image_List.shape[0], 28, 28, 1)

print(flat_Image_List[1])
flat_Image_Array.shape

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
# For encoding categorical variables and pre processing.
import sklearn.preprocessing as pre

encoder = pre.LabelBinarizer()
encoder.fit(image_Lbl_list)
outputs = encoder.transform(image_Lbl_list)

outputs[0]

inputs = flat_Image_Array.reshape(100, 49152)
inputs.shape

In [ ]:
# ------- MODEL -------
# Import keras.
import keras as kr
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dropout 

# Start a neural network, building it by layers.
model = kr.models.Sequential()
# Add a hidden layer with 784 neurons.
model.add(kr.layers.Dense(units=784, activation='relu', input_dim=49152))
# Add a hidden layer with 455 neurons.
model.add(kr.layers.Dense(units=455, activation='relu'))
# Add a hidden layer with 120 neurons.
model.add(kr.layers.Dense(units=120, activation='linear'))
# Add a hidden layer with 50 neurons.
model.add(kr.layers.Dense(units=50, activation='relu'))

# Add a three neuron output layer.
model.add(kr.layers.Dense(units=10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
flat_Image_Array = np.array(list(flat_Image_List[:])).reshape(100, 49152).astype(np.uint8)
image_Lbl_list =  np.array(list(image_Lbl_list[:])).astype(np.uint8)

flat_Image_Array[1]
#print(image_Lbl_list)

In [ ]:
# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Number of Epoch is the amount of times the training set is put through the model
# The batch size is the amount of images the models processes at one time
model.fit(inputs, outputs, epochs=10, batch_size=10)

In [ ]:
flat_Image_Array = np.array(list(flat_Image_List[:])).reshape(100, 49152).astype(np.uint8)
image_Lbl_list =  np.array(list(image_Lbl_list[:])).astype(np.uint8)